# Sentiment Classification Project

In [1]:
!pip install -r reqs.txt

Defaulting to user installation because normal site-packages is not writeable


In [ ]:
# import pytorch_lightning as L
# from pytorch_lightning.loggers import WandbLogger
# from pytorch_lightning.callbacks import LearningRateMonitor, ModelCheckpoint

import torch
import torch.nn as nn

from dataset.twitter_dataset import TwitterDataModule
# from recipes.sentiment_analysis import SentimentAnalysisNet

2023-07-03 20:56:46.052564: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [1]:
from pytorch_lightning.loggers import WandbLogger

2023-07-03 20:07:18.791882: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [2]:
batch_size = 32
embedding_dim = 300
    
from string import punctuation 
from preprocessing.embeddings import create_w2v_embeddings

translator = str.maketrans('','', punctuation)
dm = TwitterDataModule(
    "twitter-datasets/train_pos_full.txt",
    "twitter-datasets/train_neg_full.txt",
    "twitter-datasets/test_data.txt",
    convert_to_features=create_w2v_embeddings,
    convert_to_features_kwargs={
        "workers": 8,
        "vector_size": embedding_dim,
        "min_count": 1,
        "window": 5,
        "sample": 1e-3,
    },
    tokenizer=lambda x: [tweet.translate(translator).split() for tweet in x],
    batch_size=batch_size,
)

In [ ]:
dm.setup(stage="fit")
print(dm.dims)

1250000it [00:00, 1488044.29it/s]
1250000it [00:00, 1918501.17it/s]


In [2]:
from models.rnn import RNNClassifier

model = RNNClassifier(
    rnn=nn.LSTM(
        input_size=dm.dims[-1],
        hidden_size=64,
        num_layers=2,
        batch_first=True,
        dropout=0.1,
    ),
    classifier=nn.Sequential(
        nn.Linear(64, 16),
        nn.BatchNorm1d(16),
        nn.ReLU(),
        nn.Linear(16, 8),
        nn.BatchNorm1d(8),
        nn.ReLU(),
        nn.Linear(8, 1),
        nn.Sigmoid(),
    )
)

NameError: name 'nn' is not defined

In [ ]:
net = SentimentAnalysisNet(
    model,
    lr=10e-3,
)

In [ ]:
# wandb_logger = WandbLogger(project="cil")
# lr_monitor = LearningRateMonitor(logging_interval="step")
# checkpoint_callback = ModelCheckpoint(
#     monitor='valid/loss',
#     dirpath='wandb/ckp',
#     filename='models-{epoch:02d}-{valid_loss:.2f}',
#     save_top_k=3,
#     mode='min'
# )
# trainer_params = {"callbacks": [lr_monitor, checkpoint_callback]}

trainer = L.Trainer(
    max_epochs=5,
    #     # callbacks=trainer_params["callbacks"],
    #     # logger=wandb_logger,
)

In [62]:
print("start training...")
trainer.fit(model=net, datamodule=dm)
print("done!")

start training...


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type           | Params
---------------------------------------------
0 | model     | RNNClassifier  | 102 K 
1 | criterion | BCELoss        | 0     
2 | accuracy  | BinaryAccuracy | 0     
---------------------------------------------
102 K     Trainable params
0         Non-trainable params
102 K     Total params
0.410     Total estimated model params size (MB)
SLURM auto-requeueing enabled. Setting signal handlers.


Training: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=1` reached.


done!
